In [10]:
import csv
import sys
import pandas as pd
import os
import glob
from collections import Counter
import numpy as np
import pandas as pd
import glob
import csv
import sys
import os
import math
import tensorflow as tf

# ingore gpu
tf.config.set_visible_devices([], 'GPU')

import keras
from keras.preprocessing import sequence
from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Flatten, Embedding, LSTM
from keras.datasets import imdb
from keras.callbacks import EarlyStopping
from sklearn import metrics
from keras.layers.core import Dense, Activation
from keras.callbacks import ModelCheckpoint
from numpy import insert
from sklearn import preprocessing
from collections import Counter
from numpy import array
from numpy import hstack
from keras.preprocessing.sequence import TimeseriesGenerator
import pandas as pd


input_file = '/home/daniar/flashnet/model_collection/5_block_prefetching/dataset/iotrace/alibaba.cut.per_10k.rw_80_20.723/read_io.trace'
# input_file = '/home/daniar/flashnet/model_collection/5_block_prefetching/simulate_basic_prefetcher/fara_saved_models/sample_trace.txt'
# ts_record,dev_num,offset,size,io_type

df = pd.read_csv(input_file, sep=',')

# Their method

df['IOSize_log'] = np.log2(df['size'])
df['IOSize_log_roundoff']= round(df['IOSize_log'])


# print(len(Counter(df['IOSize'])))
# print(len(Counter(df['IOSize_log'])))
# print(len(Counter(df['IOSize_log_roundoff'])))

# Cell

df['ByteOffset_Delta'] = df['offset'] - df['offset'].shift(-1)
df = df.drop(df.index[-1])

# Cell

df['ByteOffset_Delta'] = df['ByteOffset_Delta'].fillna(0)

a = df['ByteOffset_Delta'].unique().tolist()

# Reverse map 1
rev_map_1 = {}
operation_id_map = {}
for i,id in enumerate(a): 
    operation_id_map[id] = i 
    rev_map_1[i] = id
df['ByteOffset_Delta_class'] = df['ByteOffset_Delta'].map(lambda x: operation_id_map[x])


from collections import Counter
x = Counter(df['ByteOffset_Delta_class'])
vals = {}
vals =  x.most_common(1000) # list of tuples 
bo_list = [] # Top 1000 deltas 

for x in vals:
    bo_list.append(x[0])
        
count = 0
label_list = []

while (count < len(df)):
    x = df['ByteOffset_Delta_class'].iloc[count]
    raw_delta = df['ByteOffset_Delta'].iloc[count]
    if x in bo_list:
        label_list.append(x)
    else:
        operation_id_map[raw_delta] = 999999
        label_list.append(999999)
    count= count + 1
    
ByteOffset_Delta_class_backup  = df['ByteOffset_Delta_class'] 
df['ByteOffset_Delta_class']  = label_list

# Reverse map 2
a = df['ByteOffset_Delta_class'].unique().tolist()
bo_map = {}
rev_map_2 = {}
for i,id in enumerate(a): 
    rev_map_2[i] = id
    bo_map[id] = i 

df['ByteOffset_Delta_Class_1001'] = df['ByteOffset_Delta_class'].map(lambda x: bo_map[x])

# My addition forward_map that combines the two small maps
forward_map = {}
count = 0
while(count < len(df)):
    raw_delta = df['ByteOffset_Delta'].iloc[count]
    forward_map[raw_delta] = bo_map[operation_id_map[raw_delta]]
    count += 1

# Check i
df['new_1001'] = df['ByteOffset_Delta'].map(lambda x: forward_map[x])
print("Check new_1001 equals 1001 classes", df['new_1001'].equals(df['ByteOffset_Delta_Class_1001']))


# CHeck answer (PASSED)
# rev_col = []
# for value in df['ByteOffset_Delta_Class_1001']:
#     if rev_map_2[value] == 999999:
#         rev_col.append(np.nan)
#     else:
#         rev_col.append(rev_map_1[rev_map_2[value]])
# df['rev_col'] = rev_col

# label_list = df['ByteOffset_Delta_Class_1001']     
# df['ByteOffset_Delta_Class_1001']  = label_list
# df = df.drop(df.index[0]) # Problem

a = df['IOSize_log_roundoff'].unique().tolist()
size_map = {}
rev_map_3 = {}
for i,id in enumerate(a):
    rev_map_3[i] = id
    size_map[id] = i 
df['Size_Class'] = df['IOSize_log_roundoff'].map(lambda x: size_map[x])


print("Delta classes unique", df['ByteOffset_Delta_Class_1001'].nunique())

# Save the three maps
import pickle
# with open('./fara_saved_models/rev_map_1.pkl', 'wb') as file:
#     pickle.dump(rev_map_1, file)
# with open('./fara_saved_models/rev_map_2.pkl', 'wb') as file:
#     pickle.dump(rev_map_2, file)
# with open('./fara_saved_models/rev_map_3.pkl', 'wb') as file:
#     pickle.dump(rev_map_3, file)

# print("reverse maps saved")
# df.head(5)

Check new_1001 equals 1001 classes True
Delta classes unique 1001


In [9]:
df.head(10)

,ts_record,dev_num,offset,size,io_type,IOSize_log,IOSize_log_roundoff,ByteOffset_Delta,ByteOffset_Delta_class,new_1001,ByteOffset_Delta_Class_1001,Size_Class
0,0,46,6656241664,4096,1,12.0,12.0,-1.421312e+06,0,0,0,0
1,1,46,6657662976,16384,1,14.0,14.0,-2.048000e+04,1,1,1,1
2,1,46,6657683456,4096,1,12.0,12.0,4.096000e+03,2,2,2,0
3,1,46,6657679360,4096,1,12.0,12.0,5.670576e+09,3,3,3,0
4,2,46,987103232,16384,1,14.0,14.0,-1.187840e+05,4,4,4,1
5,3,46,987222016,8192,1,13.0,13.0,4.096000e+03,2,2,2,2
6,3,46,987217920,4096,1,12.0,12.0,-5.659181e+09,5,5,5,0
7,4,46,6646398976,16384,1,14.0,14.0,-2.867200e+04,6,6,6,1
8,4,46,6646427648,8192,1,13.0,13.0,4.096000e+03,2,2,2,2
9,5,46,6646423552,4096,1,12.0,12.0,2.205790e+09,7,7,7,0


In [9]:
df.head(10)
columns_to_keep = [
    'offset',
    'size',
    'IOSize_log_roundoff',
    'ByteOffset_Delta_Class_1001',
    'Size_Class'
]
df = df[columns_to_keep]

## Start training LSTM

In [2]:
# Split to train, validate and test
import math
training_pt_1 = math.floor((len(df)*0.75)) 

lba_train =df[:training_pt_1]['ByteOffset_Delta_Class_1001'].tolist()
lba_test = df[training_pt_1+1:]['ByteOffset_Delta_Class_1001'].tolist()
size_train = df[:training_pt_1]['Size_Class'].tolist()
size_test = df[training_pt_1+1:]['Size_Class'].tolist()

lba_train= np.array(lba_train).reshape(-1,1) # each row only one element
lba_test= np.array(lba_test).reshape(-1,1)
size_train= np.array(size_train).reshape(-1,1)
size_test= np.array(size_test).reshape(-1,1)

# Each x is [1, ..., 32] and y is [33, ..., 65] (using 32 words to predict the next 32 words)
def create_dataset2(dataset, window_size):
    dataX, dataY = [], []
    for i in range(len(dataset) - 2 * window_size):
        a = dataset[i:(i + window_size), 0] # a is [1,2,3,4 ...], next [2,3,4,5 ...]
        dataX.append(a)
        b = dataset[(i + window_size):(i + 2* window_size), 0]
        dataY.append(b) # 
    return np.array(dataX), np.array(dataY)

lstm_num_timesteps = 32
    
X_train_lba, y_train_lba = create_dataset2(lba_train, lstm_num_timesteps)
X_test_lba, y_test_lba = create_dataset2(lba_test, lstm_num_timesteps)
X_train_size, y_train_size = create_dataset2(size_train, lstm_num_timesteps)
X_test_size, y_test_size = create_dataset2(size_test, lstm_num_timesteps)

lstm_num_features = 1
lstm_predict_sequences = True
lstm_num_predictions = 32

# X_train = np.reshape(X_train, (X_train.shape[0], lstm_num_timesteps, lstm_num_features))
# X_test = np.reshape(X_test, (X_test.shape[0], lstm_num_timesteps, lstm_num_features))
    
y_train_lba = np.reshape(y_train_lba, (y_train_lba.shape[0], lstm_num_predictions, lstm_num_features))
y_test_lba = np.reshape(y_test_lba, (y_test_lba.shape[0], lstm_num_predictions, lstm_num_features))
y_train_size = np.reshape(y_train_size, (y_train_size.shape[0], lstm_num_predictions, lstm_num_features))
y_test_size = np.reshape(y_test_size, (y_test_size.shape[0], lstm_num_predictions, lstm_num_features))                        

print(f'X_train_lba {X_train_lba.shape}, X_test_lba {X_test_lba.shape}, X_train_size {X_train_size.shape}, X_test_size {X_test_size.shape}')
print(f'y_train_lba {y_train_lba.shape}, y_test_lba {y_test_lba.shape}, y_train_size {y_train_size.shape}, y_test_size {y_test_size.shape}')

X_train_lba (6007, 32), X_test_lba (1959, 32), X_train_size (6007, 32), X_test_size (1959, 32)
y_train_lba (6007, 32, 1), y_test_lba (1959, 32, 1), y_train_size (6007, 32, 1), y_test_size (1959, 32, 1)


In [13]:
maxlen= 32

# # define two sets of inputs
# inputA = Input(shape=(32,))
# inputB = Input(shape=(32,))
# # inputA = Sequential()
# # inputB = Sequential()
vocabulary_1 = df['ByteOffset_Delta_Class_1001'].nunique()
vocabulary_2 = df['Size_Class'].nunique()

print("vocab 1 Delta_Class", vocabulary_1)
print("vocab 2 Size_class", vocabulary_2)

hidden_size = 200 # original 1500

# input=Input(shape=(no_docs,maxlen),dtype='float64')
inputA=Input(shape=(maxlen,),dtype='float64')  
inputB=Input(shape=(maxlen,),dtype='float64') 


# the first branch operates on the first input
x = Embedding(input_dim=vocabulary_1,output_dim=hidden_size,input_length=maxlen)(inputA)
x = Model(inputs=inputA, outputs=x)

# # the second branch opreates on the second input
y = Embedding(input_dim=vocabulary_2,output_dim=hidden_size,input_length=maxlen)(inputB)
y = Model(inputs=inputB, outputs=y)
# combine the output of the two branches
combined = keras.layers.concatenate([x.output, y.output])

lstm1 = LSTM(hidden_size,return_sequences=True)(combined)
lstm2 = LSTM(hidden_size, return_sequences=True)(lstm1)

# create classification output
offset = keras.layers.wrappers.TimeDistributed(Dense(units=vocabulary_1, activation='softmax'), name='offset')(lstm2)
iosize = keras.layers.wrappers.TimeDistributed(Dense(units=vocabulary_2, activation='softmax'), name='iosize')(lstm2)

model =Model([inputA,inputB],[offset,iosize]) # combining all into a Keras model

model.compile(optimizer='rmsprop',
              loss={'offset': 'sparse_categorical_crossentropy', 'iosize': 'sparse_categorical_crossentropy'},
              loss_weights={'offset': 2., 'iosize': 1.5},
              metrics={ 'offset': 'categorical_accuracy', 'iosize': 'categorical_accuracy'})
model.summary()

vocab 1 Delta_Class 1001
vocab 2 Size_class 7
Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 32)]         0           []                               
                                                                                                  
 input_6 (InputLayer)           [(None, 32)]         0           []                               
                                                                                                  
 embedding_4 (Embedding)        (None, 32, 200)      200200      ['input_5[0][0]']                
                                                                                                  
 embedding_5 (Embedding)        (None, 32, 200)      1400        ['input_6[0][0]']                
                                              

In [14]:
# Train
import time
num_epochs = 10
batch_size = 32

data_path = r'/home/aakashkapoor103/anaconda3/Data/Models/'
monitor = EarlyStopping(monitor='loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')
# checkpointer = ModelCheckpoint(filepath=data_path + '/src1_1_model-trained.hdf5', verbose=1)


print('Train...')
start_time = time.time()

valid = [X_test_lba,X_test_size],[y_test_lba,y_test_size]

model.fit([X_train_lba,X_train_size],[y_train_lba,y_train_size],
          verbose=1, epochs=num_epochs, callbacks=[monitor], batch_size=batch_size, validation_data=valid)

# model.load_weights('best_weights_src1.hdf5') # load weights from best model
model.save(f'./fara_saved_models/lstm_hidden_{hidden_size}_epochs_{num_epochs}.h5')
print('Done, elapsed', time.time() - start_time)



Train...
Epoch 1/10
188/188 [==============================] - 20s 91ms/step - loss: 10.0345 - offset_loss: 3.9754 - iosize_loss: 1.3891 - offset_categorical_accuracy: 0.0000e+00 - iosize_categorical_accuracy: 0.6825 - val_loss: 8.5087 - val_offset_loss: 3.1944 - val_iosize_loss: 1.4132 - val_offset_categorical_accuracy: 0.0000e+00 - val_iosize_categorical_accuracy: 0.8590
Epoch 2/10
188/188 [==============================] - 16s 87ms/step - loss: 8.8658 - offset_loss: 3.4807 - iosize_loss: 1.2695 - offset_categorical_accuracy: 0.0000e+00 - iosize_categorical_accuracy: 0.6520 - val_loss: 8.5810 - val_offset_loss: 3.2107 - val_iosize_loss: 1.4397 - val_offset_categorical_accuracy: 0.0000e+00 - val_iosize_categorical_accuracy: 0.7895
Epoch 3/10
188/188 [==============================] - 17s 91ms/step - loss: 7.8472 - offset_loss: 3.0650 - iosize_loss: 1.1449 - offset_categorical_accuracy: 0.0000e+00 - iosize_categorical_accuracy: 0.6114 - val_loss: 8.7388 - val_offset_loss: 3.2614 - val_

In [3]:
# Inference
hidden_size = 200
num_epochs = 10
new_model = keras.models.load_model(f'./fara_saved_models/lstm_hidden_{hidden_size}_epochs_{num_epochs}.h5')
pred1,pred2 = new_model.predict([X_test_lba,X_test_size],verbose =1 )

print("pred1 pred2 shape", pred1.shape, pred2.shape)


2023-09-18 11:57:23.009551: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


62/62 [==============================] - 3s 32ms/step
pred1 pred2 shape (1959, 32, 1001) (1959, 32, 7)


In [12]:

# Calculate accuracy


# Create data
training_pt_1 = math.floor((len(df)*0.75)) 

lba_train =df[:training_pt_1]['ByteOffset_Delta_Class_1001'].tolist()
lba_test = df[training_pt_1+1:]['ByteOffset_Delta_Class_1001'].tolist()
size_train = df[:training_pt_1]['Size_Class'].tolist()
size_test = df[training_pt_1+1:]['Size_Class'].tolist()

# ===

from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

lba_test_final = lba_test[-(len(pred1)):]
lba_size_final = size_test[-(len(pred1)):]

max_lba_accuracy = 0
max_lba_accuracy_pos = 0

max_size_accuracy = 100000
max_size_accuracy_pos = 0

pred_1 = pred1[:,0,:] # select the first prediction (1959, 1)
pred_2 = pred2[:,0,:] # select the first prediction (1959, 1)
pred_1 = np.argmax(pred_1, axis=1)
pred_2 = np.argmax(pred_2, axis=1)

print("Trans pred_1 shape", len(pred_1), "test_final shape", len(lba_test_final))
for i in range(len(pred_1)):
    print(f"i {i}, pred_addr {pred_1[i]}, pred_size {pred_2[i]}")
                 

lba_accuracy = accuracy_score(lba_test_final, pred_1)
size_accuracy = accuracy_score(lba_size_final, pred_2)

print("IO Size Accuracy", str(size_accuracy))
print("Best LBA Delta Accuracy", str(lba_accuracy))

Trans pred_1 shape 1959 test_final shape 1959
i 0, pred_addr 819, pred_size 0
i 1, pred_addr 819, pred_size 1
i 2, pred_addr 819, pred_size 0
i 3, pred_addr 819, pred_size 0
i 4, pred_addr 819, pred_size 0
i 5, pred_addr 819, pred_size 0
i 6, pred_addr 819, pred_size 0
i 7, pred_addr 819, pred_size 1
i 8, pred_addr 819, pred_size 0
i 9, pred_addr 819, pred_size 0
i 10, pred_addr 819, pred_size 0
i 11, pred_addr 819, pred_size 0
i 12, pred_addr 819, pred_size 0
i 13, pred_addr 819, pred_size 1
i 14, pred_addr 819, pred_size 0
i 15, pred_addr 819, pred_size 1
i 16, pred_addr 819, pred_size 0
i 17, pred_addr 819, pred_size 0
i 18, pred_addr 819, pred_size 0
i 19, pred_addr 819, pred_size 0
i 20, pred_addr 819, pred_size 0
i 21, pred_addr 819, pred_size 1
i 22, pred_addr 819, pred_size 0
i 23, pred_addr 819, pred_size 0
i 24, pred_addr 819, pred_size 1
i 25, pred_addr 819, pred_size 0
i 26, pred_addr 819, pred_size 0
i 27, pred_addr 819, pred_size 0
i 28, pred_addr 819, pred_size 0
i 29, p

In [ ]:
# Record
'''
=== Test 1
hidden_size = 150
num_epochs = 10
Done, elapsed 150

IO Size Accuracy 0.39
Best LBA Delta Accuracy 0.35

=== Test 2, same acc but slower
hidden_size = 150
num_epochs = 30
Done, elapsed 1019.8938829898834

IO Size Accuracy 0.4058192955589586
Best LBA Delta Accuracy 0.35324144971924454
'''

In [32]:
# DO NOT RUN (My Version)

input_file = '/home/daniar/flashnet/model_collection/5_block_prefetching/dataset/iotrace/alibaba.cut.per_10k.rw_80_20.723/read_io.trace'
# ts_record,dev_num,offset,size,io_type

df = pd.read_csv(input_file, sep=',')

# Step 1: Convert address delta into 1001 classes  
df['ByteOffset_Delta'] = df['offset'] - df['offset'].shift(-1)
df = df.drop(df.index[-1]) # drop the last row
df['ByteOffset_Delta'] = df['ByteOffset_Delta'].fillna(0)

x = Counter(df['ByteOffset_Delta'])
unique_deltas = len(x)
most_common_deltas = x.most_common(1000) # creates a map
class_map = {}
i = 0
coverage = 0
for delta, cnt in most_common_deltas:
    class_map[delta] = i
    i += 1
    coverage += cnt

df['ByteOffset_Delta_Class_1001'] = df['ByteOffset_Delta'].map(lambda x : class_map.get(x, 1001)) # If in map, assign the class. Else, assign class 1001

# Step 2: Convert io_size into classes
df['IOSize_log'] = np.log2(df['size'])
df['IOSize_log_roundoff']= round(df['IOSize_log'])
a = df['IOSize_log_roundoff'].unique().tolist()
size_map = {}
for i,id in enumerate(a): size_map[id] = i 
df['Size_Class'] = df['IOSize_log_roundoff'].map(lambda x: size_map[x])

# print("Before drop: {}".format(df.columns))
columns_to_keep = [
    'offset',
    'size',
    'IOSize_log_roundoff',
    'ByteOffset_Delta_Class_1001',
    'Size_Class'
]
df = df[columns_to_keep]
df = df.drop(df.index[0]) # Is it necessary?

print("Raw deltas", len(x), "-> Classes", df['ByteOffset_Delta_Class_1001'].nunique(), "-> Coverage", coverage / len(df))
print("Raw io_size", len(Counter(df['size'])), " -> Classes", len(Counter(df['IOSize_log_roundoff'])))

# print("Printing the df")
# df.head(10)

Raw deltas 3785 -> Classes 1000 -> Coverage 0.6560415122312825
Raw io_size 37  -> Classes 7


In [33]:
df.head(10)

,offset,size,IOSize_log_roundoff,ByteOffset_Delta_Class_1001,Size_Class
1,6657662976,16384,14.0,5,1
2,6657683456,4096,12.0,4,0
3,6657679360,4096,12.0,337,0
4,987103232,16384,14.0,43,1
5,987222016,8192,13.0,4,2
6,987217920,4096,12.0,338,0
7,6646398976,16384,14.0,9,1
8,6646427648,8192,13.0,4,2
9,6646423552,4096,12.0,339,0
10,4440633344,4096,12.0,340,0
